In [ ]:
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import MinMaxScaler
import numpy as np

app = FastAPI()


user_data = np.array([
    [1, 2],
    [1, 1], 
    [2, 3], 
    [3, 1],  
    [2, 2]  
])

# Corresponding user IDs
user_ids = np.array([101, 102, 103, 104, 105])


scaler = MinMaxScaler()
user_data_scaled = scaler.fit_transform(user_data)

# Initialize the KNN model
knn = NearestNeighbors(n_neighbors=1)
knn.fit(user_data_scaled)

class UserPreferences(BaseModel):
    country: int
    hobby: int

@app.post("/recommend/")
async def recommend_user(user_preferences: UserPreferences):
    new_user = np.array([[user_preferences.country, user_preferences.hobby]]) 
    new_user_scaled = scaler.transform(new_user)
    _, indices = knn.kneighbors(new_user_scaled)
    recommended_user_id = user_ids[indices[0][0]]
    return {"recommended_user_id": recommended_user_id}